# Temperature Control
Author: Andrei Gogosha

Import all the necessary packages for the controller.

In [1]:
from distutils.log import error
import board
import digitalio
import adafruit_max31856
import PID
import time
from datetime import datetime as dt
import csv
import matplotlib
matplotlib.use("tkAgg")
import matplotlib.pyplot as plt
import numpy as np
import keyboard
from tclab import clock, setup, Historian, Plotter
import os
import pandas
from IPython.display import clear_output
import traceback

Get current directory so we can save log files in a separate folder.

In [2]:
ROOT_DIR = os.path.realpath(os.path.join(os.path.dirname("Temperature Control.ipynb")))

Define functions to open a csv file that will be used for the temperature logs and other running information. This cell also includes a function that corrects the thermocouple temperature read by the ADAFRUIT MAX31856 amplifier using a two point calibration as described here :https://learn.adafruit.com/calibrating-sensors/two-point-calibration 

In [3]:
def log_temps(f_name,header,data):
    temp_log_w=csv.writer(log_file)
    temp_log_w.writerow(data)

def open_file(file_name,data_header):
    log_file = open(os.path.join(ROOT_DIR, 'Logs', file_name),'w',encoding='UTF8', newline='')     #Open the log file in the Log subfolder of the working directory with the specified filename
    temp_log_w = csv.writer(log_file)
    temp_log_w.writerow(data_header)     # writes the data header for the csv file
    
    return log_file     # returns the log file so it can be saved as a variable and manipulated later


def calibrated_temps(temp, TC):
    if 'HeatExB' in TC:
        RawRange =  121
        ReferenceRange = 126
        ActualTemp = (((temp + 117) * ReferenceRange) / RawRange) - 108
    if 'HeatExF' in TC:
        RawRange = 184
        ReferenceRange = 179
        ActualTemp = (((temp + 174) * ReferenceRange) / RawRange) - 161
    if 'ColdHead' in TC:
        #RawRange = 187
        #ReferenceRange = 185
        #ActualTemp = (((temp + 167) * ReferenceRange) / RawRange) - 171
        #Coldhead was consistent in its measurements, atleast at high temps, so leave it alone (pending more testing)
        ActualTemp = temp
    if 'Chamber' in TC:
        #RawRange = 35
        #ReferenceRange = 35
        #At both high and low temps, Chamber was consistently 8 degrees colder than the thermometer measured
        ActualTemp = temp + 7.6
        
    return ActualTemp

Create the temperature log files using the current date and time in the format month day year then hour and minute. This cell also creates the header for the temperature log files

In [4]:
data_f_name='Temp log {}.csv'.format(dt.now().strftime('%m-%d-%Y, %H-%M'))
data_header=['Rt', 'temp_ch', 'temp_hex_f', 'temp_hex_b', 'temp_chamber','Heat F','Heat B']
# header reads [Real time, Cold head temp, Heat exchander front temp, heat exhchanger back temp, champer temp, PID controler 1 output, Heater 1 status, PID controler 2 output, Heater 2 status]
log_file = open_file(data_f_name,data_header)

Define the coldhead, heat exchanger front and back, chamber, and heater entities that will be controled.

In [5]:
# Create sensor object, communicating over the board's default SPI bus
spi = board.SPI()

# allocate a CS pin and set the direction
cs13 = digitalio.DigitalInOut(board.D13)
cs13.direction = digitalio.Direction.OUTPUT
cs16 = digitalio.DigitalInOut(board.D16)
cs16.direction = digitalio.Direction.OUTPUT
cs25 = digitalio.DigitalInOut(board.D25)
cs25.direction = digitalio.Direction.OUTPUT
cs26 = digitalio.DigitalInOut(board.D26)
cs26.direction = digitalio.Direction.OUTPUT
HeaterF = digitalio.DigitalInOut(board.D17)
HeaterF.direction= digitalio.Direction.OUTPUT
HeaterB = digitalio.DigitalInOut(board.D18)
HeaterB.direction = digitalio.Direction.OUTPUT

# create a thermocouple object with the above pin assignements
ColdHead = adafruit_max31856.MAX31856(spi, cs13,thermocouple_type=adafruit_max31856.ThermocoupleType.T)
HeatExF = adafruit_max31856.MAX31856(spi, cs16,thermocouple_type=adafruit_max31856.ThermocoupleType.T)     #Heat exchanger thermocouple facing the coldhead
HeatExB = adafruit_max31856.MAX31856(spi, cs25,thermocouple_type=adafruit_max31856.ThermocoupleType.T)     #Heat exchanger thermocouple facing the chamber
Chamber = adafruit_max31856.MAX31856(spi, cs26,thermocouple_type=adafruit_max31856.ThermocoupleType.T)


Initiate PID controls for Front Heat Exchanger

In [6]:
HeaterF.value = False
HeatF_on=False
targetT1 = -115   #initial inputs for PID control
P1 = 0.2*0.6
I1 = 1.2*0.2/60
D1 = 3*0.2*60/40

controllerF = PID.PID(P1, I1, D1)        # create pid control
controllerF.SetPoint = targetT1             # initialize the controler
controllerF.setSampleTime(0.5)

Initiate PID controls for Back Heat Exchanger

old
P1 = 0.7*0.6
I1 = 0.7*1.2/58
D1 = 3*0.7*58/40

In [7]:
HeaterB.value = False 
HeatB_on = False
targetT2 = -94.5     #initial inputr fot PID control
P2 = 0.2*0.6
I2 = 1.2*0.2/60
D2 = 3*0.2*60/40

controllerB = PID.PID(P2, I2, D2)     #creats the pid control
controllerB.SetPoint = targetT2     #initialize the controler
controllerB.setSampleTime(0.5)

Read the coldhead, heat exchanger front and back, chamber temperatures then log them. These values are taken for initialization purposes

In [8]:
ColdHead.initiate_one_shot_measurement()
HeatExF.initiate_one_shot_measurement()
HeatExB.initiate_one_shot_measurement()
Chamber.initiate_one_shot_measurement()
temp_coldhead=calibrated_temps(ColdHead.unpack_temperature(), 'ColdHead')
temp_HeatExF=calibrated_temps(HeatExF.unpack_temperature(),'HeatExF')
temp_HeatExB=calibrated_temps(HeatExB.unpack_temperature(),'HeatExB')
temp_chamber=calibrated_temps(Chamber.unpack_temperature(), 'Chamber')
MV1 = controllerF.output # get the new pid values
MV2 = controllerB.output
HeatF_status = 0 #0=False 1=True
HeatB_status = 0 #0=False 1=True
time_stamp = dt.now().strftime('%H:%M:%S')
print(temp_coldhead, temp_HeatExF, temp_HeatExB, temp_chamber)
log_temps(data_f_name,data_header,[dt.now().strftime('%H:%M:%S'), temp_coldhead, temp_HeatExF, temp_HeatExB, temp_chamber,HeatF_status, HeatB_status])
log_file.flush() #pushes the data collected this loop to the csv.

17.9453125 17.98479959239131 24.158445247933884 17.6390625


In [9]:
Ledger=np.array([[time_stamp], [temp_coldhead], [temp_HeatExF], [temp_HeatExB], [temp_chamber], [MV1], [HeatF_status], [MV2], [HeatB_status]])
                #dtype=[('time_stamp','str'),('temp_coldhead','float'),('temp_HeatExF','float'),('temp_HeatExB','float')\
                 #      ,('temp_chamber','float'),('MV1','float'),('HeatF_status','int'),('MV2','float'),('HeatB_status','int')])

plt.ion()     #creates a floating interactive matplotlib plot 
fig, ax = plt.subplots(figsize=(12,5))     #initialize plots for the coldhead, heat exchanger front and back, chamber and put them all in one larer plot
chline,  = ax.plot(Ledger[0,:], Ledger[1,:].astype('float32'), label='Cold Head')
hexfline, = ax.plot(Ledger[0,:], Ledger[2,:].astype('float32'), label='Heat Exchanger Front')
hexbline, = ax.plot(Ledger[0,:], Ledger[3,:].astype('float32'), label='Heat Exchanger Back')
chamberline, =ax.plot(Ledger[0,:], Ledger[4,:].astype('float32'), label='Chamber')
ax.locator_params(tight=True, nbins=10)
ax.legend()
ax.set_xlabel('Time (Hour:Min)')
ax.set_ylabel('Temperature (Celsius)')

/tmp/ipykernel_30473/4263136039.py:11: UserWarning: 'set_params()' not defined for locator of type <class 'matplotlib.category.StrCategoryLocator'>
  ax.locator_params(tight=True, nbins=10)


Text(0, 0.5, 'Temperature (Celsius)')

Create the temperature plot

In [14]:
print_head=["Time" ,"Coldhead", "Heat Exchanger Front", "Heat Exchanger Back", "Chamber", "PID1 Output", "Heater17 Status", "PID2 Output", "Heater18 Status"]
try:     # try and excep statement used to catch error and log them to a specified file
    runlen=1
    itt_len=6
    while True:
        now = time.time() # keep track of when the loop starts so that we keep a consistant loop runtime 
        clear_output(wait=True)     #clears the print output from this cell 
        if os.stat(os.path.join(ROOT_DIR, 'Logs', data_f_name)).st_size>= 4194304:     #checks if the current log file is 4Mb, if it is it creates a new log file 
                data_f_name='Temp log {}.csv'.format(dt.now().strftime('%m-%d-%Y, %H-%M'))
                log_file.close()
                log_file = open_file(data_f_name, data_header)
        #reads the coldhead, heat exchanger front and back, chamber temepratures
        t1=time.time()
        ColdHead.initiate_one_shot_measurement()
        HeatExF.initiate_one_shot_measurement()
        HeatExB.initiate_one_shot_measurement()
        Chamber.initiate_one_shot_measurement()
        ColdHead._wait_for_oneshot()
        temp_coldhead=calibrated_temps(ColdHead.unpack_temperature(), 'ColdHead')
        if not HeatExF.oneshot_pending:
            temp_HeatExF=calibrated_temps(HeatExF.unpack_temperature(),'HeatExF')
        else:
            HeatExF._wait_for_oneshot()
            temp_HeatExF=calibrated_temps(HeatExF.unpack_temperature(),'HeatExF')
        if not HeatExB.oneshot_pending:
            temp_HeatExB=calibrated_temps(HeatExB.unpack_temperature(),'HeatExF')
        else:
            HeatExB._wait_for_oneshot()
            temp_HeatExB=calibrated_temps(HeatExB.unpack_temperature(),'HeatExF')
        if not Chamber.oneshot_pending:
            temp_chamber=calibrated_temps(Chamber.unpack_temperature(),'HeatExF')
        else:
            Chamber._wait_for_oneshot()
            temp_chamber=calibrated_temps(Chamber.unpack_temperature(),'HeatExF')
        t2=time.time()
        print(t2 - t1)
        #temp_coldhead=calibrated_temps(ColdHead.temperature, 'ColdHead')
        #temp_HeatExF=calibrated_temps(HeatExF.temperature,'HeatExF')
        #temp_HeatExB=calibrated_temps(HeatExB.temperature,'HeatExB')
        #temp_chamber=calibrated_temps(Chamber.temperature, 'Chamber')
        controllerF.update(temp_HeatExF) # update the pid controlers
        controllerB.update(temp_HeatExB)
        MV1 = controllerF.output # get the new pid values
        MV2 = controllerB.output
        if MV1 > 0:      #temp too low turn heater on
            HeaterF.value = True
            HeatF_status = 1
        else:     #turn heat off
            HeaterF.value = False
            HeatF_status = 0
        if MV2 > 0:     #temp too low turn heater on
            HeaterB.value = True
            HeatB_status = 1
        else:     #turn heat off
            HeaterB.value = False
            HeatB_status = 0
        time_stamp= dt.now().strftime('%H:%M:%S')     #timestamp used for x axis tick
        t3 = time.time()
        Ledger=np.append(Ledger,[[time_stamp], [temp_coldhead], [temp_HeatExF], [temp_HeatExB], [temp_chamber], [MV1], [HeatF_status], [MV2], [HeatB_status]],axis=1 )
        P_Ledger=Ledger.transpose()
        to_print=pandas.DataFrame(columns=print_head)     #using a pandas dataframe create a pretty real time readout of the temperatures and status of the heaters
        for i in P_Ledger[-20:,:]:
            to_print.loc[len(to_print.index)]=i
        print(to_print.to_markdown(tablefmt="simple_outline",floatfmt=(".2f",".2f",".2f",".2f",".3f",".3f"),numalign="right"))
        if runlen==itt_len:     #change to be however many itterations you want before updating logs
            Coldhead_avg=np.average(Ledger[1,-itt_len:].astype('float32'))
            HeatExF_avg=np.average(Ledger[2,-itt_len:].astype('float32'))
            HeatExB_avg=np.average(Ledger[3,-itt_len:].astype('float32'))
            Chamber_avg=np.average(Ledger[4,-itt_len:].astype('float32'))
            log_temps(data_f_name,data_header,[time_stamp, Coldhead_avg, HeatExF_avg, HeatExB_avg, Chamber_avg, HeatF_status , HeatB_status])     #write to temp log file
            log_file.flush() #pushes the data collected this loop to the csv.
            runlen = 0
        if len(Ledger[1])>=601:     #create a 10 minute plot of temperature data
            Ledger=np.delete(Ledger, obj=0,axis=1)
        t4=time.time()
        print(t4-t3)
        chline.set_ydata(Ledger[1,:].astype('float32'))
        chline.set_xdata(Ledger[0,:])
        hexfline.set_ydata(Ledger[2,:].astype('float32'))
        hexfline.set_xdata(Ledger[0,:])
        hexbline.set_ydata(Ledger[3,:].astype('float32'))
        hexbline.set_xdata(Ledger[0,:])
        chamberline.set_ydata(Ledger[4,:].astype('float32'))
        chamberline.set_xdata(Ledger[0,:])
        ax.set_xticks(Ledger[0,-601::50])     #10 minutes of timestamps in 100 second intervals
        ax.set_xticklabels(Ledger[0,-601::50],rotation=25)
        ax.relim()
        ax.autoscale_view()
        fig.canvas.draw()
        fig.canvas.flush_events()
        elapsed = time.time() - now # how long was it running?
        print(elapsed)
        try:time.sleep(1-elapsed)     # make the loop run every 1 seconds
        except: time.sleep(0.1)
        runlen += 1
        
#Opens the relays (stops the heaters) when program interrupted
except Exception as e:
    HeaterF.value = False
    HeatF_on = False
    HeaterB.value = False
    HeatB_on = False
    
    if e == 'KeyboardInterrupt':
        print('Interrupted')
    else:      #code to write any errors to a specified error log file in the logs subdirectory of the working directory
        if not os.path.exists(os.path.join(ROOT_DIR, 'Logs', 'Error Logs.txt')):
            with open(os.path.join(ROOT_DIR, 'Logs', 'Error Logs'), 'w', encoding='UTF-8') as file:
                file.write('')
        with open(os.path.join(ROOT_DIR, 'Logs', 'Error Logs'), 'a', encoding='UTF-8') as file:
            file.write('-------------------------------------------------'+'\n')
            file.write(dt.now().strftime('%Y-%m-%d %H:%M:%S')+'\n')
            traceback.print_exc(file=file)
            file.write('\n')
        traceback.print_exc()

0.17505574226379395
┌────┬──────────┬────────────┬────────────────────────┬───────────────────────┬───────────┬───────────────┬───────────────────┬───────────────┬───────────────────┐
│    │ Time     │   Coldhead │   Heat Exchanger Front │   Heat Exchanger Back │   Chamber │   PID1 Output │   Heater17 Status │   PID2 Output │   Heater18 Status │
├────┼──────────┼────────────┼────────────────────────┼───────────────────────┼───────────┼───────────────┼───────────────────┼───────────────┼───────────────────┤
│  0 │ 11:12:54 │      17.92 │                  18.18 │                17.954 │    18.084 │      -16.1172 │                 0 │      -13.5469 │                 0 │
│  1 │ 11:12:55 │      17.91 │                  18.00 │                18.000 │    18.046 │      -15.8767 │                 0 │      -13.6208 │                 0 │
│  2 │ 11:12:56 │      17.98 │                  18.02 │                17.863 │    18.122 │      -16.0633 │                 0 │      -13.4404 │                 


KeyboardInterrupt



In [11]:
#print(len(Ledger[1]))

Loop that reads the temperatures of the coldhead, heat exchanger front and back, chamber logs them, updates the PID control and runs the heater, then plots the tmperatures.

In [12]:
# plot_window = 1
# coldhead_temps = np.array(np.zeros([plot_window]))     #initialize numpy arrays for the temperatures of the coldhead, heat exchanger front and back, chamber
# HeatExF_temps = np.array(np.zeros([plot_window]))
# HeatExB_temps = np.array(np.zeros([plot_window]))
# chamber_temps = np.array(np.zeros([plot_window]))
# coldhead_temps[0] = temp_coldhead # Sets the first value of each plot line
# HeatExF_temps[0] = temp_HeatExF
# HeatExB_temps[0] = temp_HeatExB
# chamber_temps[0] = temp_chamber
# time_stamps = []

# for i in range(plot_window):     #creates first timestamp that will be used for the x axis tick marks
#     time_stamps.append(dt.now().strftime('%H:%M:%S'))
#     #x_var.append(time.asctime(time.time()))
# plt.ion()     #creates a floating interactive matplotlib plot 
# fig, ax = plt.subplots(figsize=(12,5))     #initialize plots for the coldhead, heat exchanger front and back, chamber and put them all in one larer plot
# chline,  = ax.plot(time_stamps, coldhead_temps, label='Cold Head')
# hexfline, = ax.plot(time_stamps, HeatExF_temps, label='Heat Exchanger Front')
# hexbline, = ax.plot(time_stamps, HeatExB_temps, label='Heat Exchanger Back')
# chamberline, =ax.plot(time_stamps, chamber_temps, label='Chamber')
# ax.locator_params(tight=True, nbins=10)
# ax.legend()
# ax.set_xlabel('Time (Hour:Min)')
# ax.set_ylabel('Temperature (Celsius)')
# # NOTE this cell raises  a matplotlib error. Ignore it, it does not appear to have any impact on the functioning of the program

In [13]:
# print_log=[]     #these two lines are used for the real time readout of the coldhead, heat exchanger front and back, chamber status
# print_head=["Time" ,"Coldhead", "Heat Exchanger Front", "Heat Exchanger Back", "Chamber", "PID1 Output", "Heater17 Status", "PID2 Output", "Heater18 Status"]
# try:     # try and excep statement used to catch error and log them to a specified file
#     runlen=0
#     itt_len=10
#     while True:
#         now = time.time() # keep track of when the loop starts so that we keep a consistant loop runtime 
#         clear_output(wait=True)     #clears the print output from this cell 
#         if os.stat(os.path.join(ROOT_DIR, 'Logs', data_f_name)).st_size>= 4194304:     #checks if the current log file is 4Mb, if it is it creates a new log file 
#                 data_f_name='Temp log {}.csv'.format(dt.now().strftime('%m-%d-%Y, %H-%M'))
#                 log_file.close()
#                 log_file = open_file(data_f_name, data_header)
#         temp_coldhead=calibrated_temps(ColdHead.temperature, 'ColdHead')     #reads the coldhead, heat exchanger front and back, chamber temepratures
#         temp_HeatExF=calibrated_temps(HeatExF.temperature,'HeatExF')
#         temp_HeatExB=calibrated_temps(HeatExB.temperature,'HeatExB')
#         temp_chamber=calibrated_temps(Chamber.temperature, 'Chamber')
#         controllerF.update(temp_HeatExF) # update the pid controlers
#         controllerB.update(temp_HeatExB)
#         MV1 = controllerF.output # get the new pid values
#         MV2 = controllerB.output
#         if MV1 > 0:      #temp too low turn heater on
#             HeaterF.value = True
#             HeatF_status = 1
#         else:     #turn heat off
#             HeaterF.value = False
#             HeatF_status = 0
#         if MV2 > 0:     #temp too low turn heater on
#             HeaterB.value = True
#             HeatB_status = 1
#         else:     #turn heat off
#             HeaterB.value = False
#             HeatB_status = 0
#         coldhead_temps = np.append(coldhead_temps, temp_coldhead)     #update numpy arrays of temps
#         HeatExF_temps = np.append(HeatExF_temps, temp_HeatExF)
#         HeatExB_temps = np.append(HeatExB_temps, temp_HeatExB)
#         chamber_temps = np.append(chamber_temps, temp_chamber)
#         time_stamps.append(dt.now().strftime('%H:%M:%S'))     #timestamp used for x axis tick
#         if runlen==itt_len:     #change to be however many itterations you want before updating logs
#             Coldhead_avg=np.avg(coldhead_temps[-itt_avg:])
#             HeatExF=np.avg(HeatExF_temps[-itt_avg:])
#             HeatExB=np.avg(HeatExB_temps[-itt_avg:])
#             Chamber=np.avg(chamber_temps[-itt_avg:])
#             log_temps(data_f_name,data_header,[dt.now().strftime('%Y-%m-%d %H:%M:%S'), temp_coldhead, temp_HeatExF, temp_HeatExB, temp_chamber, HeatF_status , HeatB_status])     #write to temp log file
#             log_file.flush() #pushes the data collected this loop to the csv.
#             runlen = 0
#         log_temps(data_f_name,data_header,[dt.now().strftime('%Y-%m-%d %H:%M:%S'), temp_coldhead, temp_HeatExF, temp_HeatExB, temp_chamber, HeatF_status , HeatB_status])     #write to temp log file
#         log_file.flush() #pushes the data collected this loop to the csv.
#         if len(print_log)<=20:     #adds data to the print log if the log is less than 20 entries otherwise remove the oldest and then updats the log
#             print_log.append([dt.now().strftime('%Y-%m-%d %H:%M:%S'), temp_coldhead, temp_HeatExF, temp_HeatExB, temp_chamber, HeatF_status , HeatB_status])
#         else:
#             print_log.pop(0)
#             print_log.append([dt.now().strftime('%Y-%m-%d %H:%M:%S'), temp_coldhead, temp_HeatExF, temp_HeatExB, temp_chamber, MV1, HeatF_status, MV2, HeatB_status])
#         to_print=pandas.DataFrame(print_log,index=None, columns=print_head)     #using a pandas dataframe create a pretty real time readout of the temperatures and status of the heaters
        
#         print(to_print.to_markdown(tablefmt="simple_outline",floatfmt=(".2f",".2f",".2f",".2f",".3f",".3f"),numalign="right"))
#         if len(coldhead_temps)<=300:     #create a 10 minute plot of temperature data
#             chline.set_ydata(coldhead_temps)
#             chline.set_xdata(time_stamps)
#             hexfline.set_ydata(HeatExF_temps)
#             hexfline.set_xdata(time_stamps)
#             hexbline.set_ydata(HeatExB_temps)
#             hexbline.set_xdata(time_stamps)
#             chamberline.set_ydata(chamber_temps)
#             chamberline.set_xdata(time_stamps)
#         else:# if the program has been running for longer than 10 minuts take the last 10 minuts of data for the plot
#             chline.set_ydata(coldhead_temps[-300:])
#             chline.set_xdata(time_stamps[-300:])
#             hexfline.set_ydata(HeatExF_temps[-300:])
#             hexfline.set_xdata(time_stamps[-300:])
#             hexbline.set_ydata(HeatExB_temps[-300:])
#             hexbline.set_xdata(time_stamps[-300:])
#             chamberline.set_ydata(chamber_temps[-300:])
#             chamberline.set_xdata(time_stamps[-300:])
#         #ax.set_ylim(0,24)
#         ax.set_xticks(time_stamps[-301::50])     #10 minutes of timestamps in 100 second intervals
#         ax.set_xticklabels(time_stamps[-301::50],rotation=25)
#         ax.relim()
#         ax.autoscale_view()
#         fig.canvas.draw()
#         fig.canvas.flush_events()
#         elapsed = time.time() - now  # how long was it running?
#         try:time.sleep(1.-elapsed)     # make the loop run every 2 seconds
#         except: time.sleep(0.5)
#         runlen += 1
        
# #Opens the relays (stops the heaters) when program interrupted
# except Exception as e:
#     HeaterF.value = False
#     HeatF_on = False
#     HeaterB.value = False
#     HeatB_on = False
    
#     if e == 'KeyboardInterrupt':
#         print('Interrupted')
#     else:      #code to write any errors to a specified error log file in the logs subdirectory of the working directory
#         if not os.path.exists(os.path.join(ROOT_DIR, 'Logs', 'Error Logs.txt')):
#             with open(os.path.join(ROOT_DIR, 'Logs', 'Error Logs'), 'w', encoding='UTF-8') as file:
#                 file.write('')
#         with open(os.path.join(ROOT_DIR, 'Logs', 'Error Logs'), 'a', encoding='UTF-8') as file:
#             file.write('-------------------------------------------------'+'\n')
#             file.write(dt.now().strftime('%Y-%m-%d %H:%M:%S')+'\n')
#             traceback.print_exc(file=file)
#             file.write('\n')
#         traceback.print_exc()